In [1]:
from IPython.display import HTML

# Cell visibility - COMPLETE:
#tag = HTML('''<style>
#div.input {
#    display:none;
#}
#</style>''')
#display(tag)

#Cell visibility - TOGGLE:
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<p style="text-align:right">
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.</p>''')
display(tag)

## Polinomok deriválása

Az interaktív példa segítségével közelebbről vizsgálható a polinomiális függvények deriválása. Szabadon megválaszthatóak a polinom együtthatók (a negyedik hatványig), a függvény pedig megjelenik az első deriváltjával együtt az ábrán. Mind a kiinduló függvény, mind pedig a deriváltja megjelenik képletek formájában is.

In [2]:
%matplotlib inline
#%config InlineBackend.close_figures=False 
from ipywidgets import interactive
from ipywidgets import widgets
from IPython.display import Latex, display, Markdown # For displaying Markdown and LaTeX code
import matplotlib.pyplot as plt
import numpy as np
import math
import sympy as sym
import matplotlib.patches as mpatches
from IPython.display import HTML, clear_output
from IPython.display import display


# Mathematical notation of a specific (user-defined) polynomial, shown as Markdown
fourth_order = "e + d * x + c * x ** 2 + b * x ** 3 + a * x ** 4"
third_order = "d + c * x + b * x ** 2 + a * x ** 3"
second_order = "c + b * x + a * x ** 2"
first_order = "b + a * x"
zero_order = "a"
tf = sym.sympify(fourth_order)  
w_mark = Markdown('$%s$' %sym.latex(tf))


# General mathematical notation of a polynomial (shown in Label widget)
fourth_order_html = "$f(x)=ax^4$ + $bx^3$ + $cx^2$ + $dx$ + $e$"
third_order_html = "$f(x)=ax^3$ + $bx^2$ + $cx$ + $d$"
second_order_html = "$f(x)=ax^2$ + $bx$ + $c$"
first_order_html = "$f(x)=ax$ + $b$"
zero_order_html = "$f(x)=a$"
w_funLabel = widgets.Label(layout=widgets.Layout(width='40%', margin='0px 0px 0px 50px'),)


# Input sliders for coefficients of a polynomial
fs_a = widgets.FloatSlider(description='$a$', min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_b = widgets.FloatSlider(description='$b$',min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_c = widgets.FloatSlider(description='$c$',min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_d = widgets.FloatSlider(description='$d$',min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_e = widgets.FloatSlider(description='$e$',min=-10.0, max=10.0, step=0.5, continuous_update=False)
input_widgets = widgets.HBox([fs_a, fs_b, fs_c, fs_d, fs_e])


# Dropdown: selecting the order of a ploynomial [0-4]
# Selection will invoke showing/hiding particular slider widgets
dd_order = widgets.Dropdown(
    options=['4', '3', '2', '1', '0'],
    value='4',
    description='Adja meg a polinom fokszámát [0-4]:',
    disabled=False,
    style = {'description_width': 'initial'},
)

def dropdown_eventhandler(change):
    fs_a.layout.visibility = 'hidden'
    fs_b.layout.visibility = 'hidden'
    fs_c.layout.visibility = 'hidden'
    fs_d.layout.visibility = 'hidden'
    fs_e.layout.visibility = 'hidden'
    
    if (dd_order.value == '4'):
        fs_a.layout.visibility = 'visible'
        fs_a.description = '$a$'
        fs_b.layout.visibility = 'visible'
        fs_b.description = '$b$'
        fs_c.layout.visibility = 'visible'
        fs_c.description = '$c$'
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$d$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$e$'
        w_funLabel.value=fourth_order_html
    if (dd_order.value == '3'):      
        fs_a.value = 0
        fs_b.layout.visibility = 'visible'
        fs_b.description = '$a$'
        fs_c.layout.visibility = 'visible'
        fs_c.description = '$b$'
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$c$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$d$'
        w_funLabel.value=third_order_html
    if (dd_order.value == '2'):
        fs_a.value = 0
        fs_b.value = 0
        fs_c.layout.visibility = 'visible'
        fs_c.description = '$a$'
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$b$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$c$'
        w_funLabel.value=second_order_html
    if (dd_order.value == '1'):
        fs_a.value = 0
        fs_b.value = 0
        fs_c.value = 0
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$a$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$b$'
        w_funLabel.value=first_order_html
    if (dd_order.value == '0'):
        fs_a.value = 0
        fs_b.value = 0
        fs_c.value = 0
        fs_d.value = 0
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$a$'
        w_funLabel.value=zero_order_html

dd_order.observe(dropdown_eventhandler, names='value')


# Utility functions
x = sym.symbols('x')

def fprime(fx):
    if not fx.is_zero:
        return sym.diff(fx, x)
    else:
        return ""

def convert(base_text, ss):
    if ss != "":
        tf = sym.sympify(ss)  
        display(Markdown(base_text + '$%s$' %sym.latex(tf)))
        
def polynomial_function(X_quaded, X_cubed, X_squared, X, const, x):
    return const + X * x + X_squared * x ** 2 + X_cubed * x ** 3 + X_quaded * x ** 4

def fun(x):
    global a, b, c, d, e
    return e + d * x + c * x **2 + b * x ** 3 + a * x ** 4


#Plot
red_patch = mpatches.Patch(color='red', label='$f(x)$')
blue_patch = mpatches.Patch(color='green', label='$f(x)$ első deriváltja')

XLIM = 10
YLIM = 20
XTICK = 2
YTICK = 5
  
a = 0
b = 0
c = 0
d = 0
e = 0

def plot_function(X_quaded, X_cubed, X_squared, X, const):
    
        global a, b, c, d, e, x
        a = X_quaded
        b = X_cubed
        c = X_squared
        d = X
        e = const
        fig = plt.figure(figsize=(12,6))

        x_p = np.linspace(-XLIM, XLIM, num=1000)
        plt.plot(x_p, polynomial_function(X_quaded, X_cubed, X_squared, X, const, x_p), 'r-')
        y_p = [polynomial_function(X_quaded, X_cubed, X_squared, X, const, num) for num in x_p]
        dydx = np.diff(y_p) / np.diff(x_p)
        plt.plot(x_p[:-1], dydx, 'g-')
        plt.grid(True)
        plt.xlabel('x')
        plt.ylabel('$f(x)$ és deriváltja')
        plt.xlim(-XLIM, XLIM)
        plt.ylim(-YLIM, YLIM)
        plt.axhline(y=0,lw=0.8,color='k')
        plt.axvline(x=0,lw=0.8,color='k')
        plt.legend(handles=[red_patch, blue_patch])
        plt.show()

        convert("Bemeneti függvény $f(x)$: ", fun(x))
        deriv = fprime(fun(x))
        convert("$f(x)$ első deriváltja: ", deriv)

# Show and interact
w_funLabel.value=fourth_order_html
control_widgets = widgets.HBox()
control_widgets.children=[dd_order, w_funLabel]
display(control_widgets)
interactive(plot_function, const=fs_e, X=fs_d, X_squared=fs_c, X_cubed=fs_b, X_quaded=fs_a)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='$a$', max=10.0, min=-10.0, …